# Linear Factor Models

In [1]:
import Formatting
include("jlFiles/printmat.jl")

include("jlFiles/OlsFn.jl")
include("jlFiles/NWFn.jl")

NWFn (generic function with 1 method)

In [2]:
xx   = readdlm("Data/FFmFactorsPs.csv",',',header=true)
x    = xx[1]
Rme  = x[:,2]
RSMB = x[:,3]                #small minus big firms
RHML = x[:,4]                #high minus low book-to-market ratio
Rf   = x[:,5]                #interest rate


x  = readdlm("Data/FF25Ps.csv",',')  #no header line: x is matrix
R  = x[:,2:end]                      #returns for 25 FF portfolios
Re = R .- Rf                         #excess returns for the 25 FF portfolios

(T,n) = size(Re,1,2)                 #no. obs and  no. test assets

(388,25)

## Testing alpha from Time Series Regressions


With $n$ test assets and $K$ excess return factors, the linear factor model is
$$
R_{t}^{e}=\alpha+\beta f_{t}+\varepsilon_{t},
$$
and the testable hypothesis is that all alphas are zero. 

The moment conditions
are
$$
\operatorname*{E}g_{t}(\alpha,\beta)=\operatorname*{E}\left(  \left[
\begin{array}
[c]{c}%
1\\
f_{t}%
\end{array}
\right]  \otimes\varepsilon_{t}\right)  =\operatorname*{E}\left(  \left[
\begin{array}
[c]{c}%
1\\
f_{t}%
\end{array}
\right]  \otimes(R_{t}^{e}-\alpha-\beta f_{t})\right)  =\mathbf{0}%
_{n(1+K)\times1}.
$$


Since the model is exactly identified, the GMM covariance matrix of the
parameters ($\alpha,\beta$) is

$$
\sqrt{T}(\hat{\beta}-\beta_{0})\overset{d}{\rightarrow}N(0,V)\text{ where
}V=D_{0}^{-1}S(D_{0}^{-1})^{\prime},
$$

where $S$ is the covariance matrix of $\sqrt{T}\bar{g}_{t}$  and $D_{0}$ is
the Jacobian which here equals

$$
D_{0}=-\operatorname*{E}\left(  \left[
\begin{array}
[c]{c}%
1\\
f_{t}%
\end{array}
\right]  \left[
\begin{array}
[c]{c}%
1\\
f_{t}%
\end{array}
\right]  ^{\prime}\right)  \otimes I_{n}\text{ so }D_{0}^{-1}=-\left[
\operatorname*{E}\left(  \left[
\begin{array}
[c]{c}%
1\\
f_{t}%
\end{array}
\right]  \left[
\begin{array}
[c]{c}%
1\\
f_{t}%
\end{array}
\right]  ^{\prime}\right)  \right]  ^{-1}\otimes I_{n}%
$$

In [3]:
                                           #FF, testing alphas
cf        = [ones(T) Rme RSMB RHML]       #factors, constant and 3 FF factors
(b,epsM,) = OlsFn(Re,cf)
alfaM     = b[1:1,:]'                       #nx1

#g_    = HDirProdFn(cf,epsM)
g = fill(NaN,(T,size(cf,2)*n))            #moment conditions, regressors*residual
for t = 1:T
  g[t,:] = kron(cf[t:t,:],epsM[t:t,:])
end
S0   = NWFn(g,0)                          #ACov(sqrt(T)*gbar)
Sxx  = cf'cf/T
D0_1 = -kron(inv(Sxx),eye(n))
V    = D0_1*S0*D0_1'
WaldStat  = alfaM'inv(V[1:n,1:n]/T)*alfaM   #test if alpha=0

println("\nTesting alpha = 0, test statistic, df, and 10% critical value of Chi-square(25)")
printmat([WaldStat length(alfaM) 34.38])


Testing alpha = 0, test statistic, df, and 10% critical value of Chi-square(25)
   162.011    25.000    34.380



## Testing Cross-Sectional Regressions

The following code from uses GMM to test $\operatorname*{E}R^{e}=\beta^{\prime}\lambda$ from an FF\ model---the case of
"General Factors".

(The factors happen to be excess returns,
but that does not stop us from applying the method for general factors.) 

The code applies GMM without a weighting function. Instead, an $A$ matrix is defined
such that $A\bar{g}=\mathbf{0}$ becomes the effective moment conditions. The
matrix is chosen to make the factor risk premia ($\lambda$) the same as in the
old-fashioned cross-sectional regressions.

### Getting the Point Estimates

In [4]:
                                          #FF, testing ERe = lambda'beta
cf        = [ones(T) Rme RSMB RHML]     #factors, constant and 3 FF factors
K         = size(cf,2) - 1                #number of factors, excluding constant
(b,epsM,) = OlsFn(Re,cf)
bM        = b'                            #nx(1+K)

ERe    = mean(Re,1)'
betaM  = bM[:,2:end]
theta  = betaM'
lambda = (theta*betaM)\(theta*ERe)       #cross-sectional estimate of price of factor risk

println("lambda:")
printmat(lambda)

lambda:
     0.567
     0.252
     0.439



### Testing

In [5]:
#g_    = [HDirProdFn(cf,epsM),Re - repmat(lambda'*betaM',T,1)]
g = fill(NaN,(T,size(cf,2)*n))           #moment conditions, regressors*residual
for t = 1:T
  g[t,:] = kron(cf[t:t,:],epsM[t:t,:])
end
g = [g (Re - repmat(lambda'*betaM',T,1))]
p = length(bM) + length(lambda)             #no. parameters
q = size(g,2)                               #no. moment conditions

gbar = mean(g,1)'
println("the sample moment conditions are:")
printmat(gbar')
S0   = NWFn(g,0)

Sxx  = cf'cf/T
D0LL = kron([0 lambda'],eye(n))             #lower left of D0
D0   = - [kron(Sxx,eye(n)) zeros(n*(1+K),K);
          D0LL              betaM           ]

A    = [eye(n*(1+K))     zeros(n*(1+K),n);     #A*gbar, traditional approach
        zeros(K,n*(1+K)) theta ]

Psia = eye(q) - D0*inv(A*D0)*A
Psi3 = Psia*S0*Psia'                          #Cov[sqrt(T)*gbar], rank q-p

WaldStat  = gbar'pinv(Psi3/T)*gbar            #test of moment conditions

println("\nTesting ERe = lambda'beta, test statistic, df and 10% critical value of Chi-square(22)")
printmat([WaldStat (q-p) 30.81])

the sample moment conditions are:
     0.000    -0.000    -0.000    -0.000     0.000     0.000     0.000    -0.000     0.000    -0.000     0.000     0.000     0.000    -0.000    -0.000    -0.000    -0.000    -0.000    -0.000    -0.000    -0.000    -0.000     0.000    -0.000    -0.000    -0.000     0.000    -0.000    -0.000     0.000     0.000     0.000     0.000     0.000     0.000    -0.000     0.000     0.000    -0.000    -0.000    -0.000    -0.000    -0.000    -0.000    -0.000    -0.000    -0.000     0.000    -0.000     0.000     0.000    -0.000     0.000    -0.000     0.000     0.000     0.000    -0.000    -0.000    -0.000    -0.000     0.000     0.000    -0.000     0.000     0.000    -0.000    -0.000    -0.000    -0.000    -0.000    -0.000    -0.000    -0.000     0.000    -0.000     0.000     0.000     0.000     0.000    -0.000    -0.000    -0.000     0.000    -0.000     0.000    -0.000    -0.000     0.000     0.000     0.000     0.000     0.000     0.000     0.000     0.000    -0